In [1]:
cd '/Users/zoltan/Dropbox/Chronostratigraphy/stratigraph'

/Users/zoltan/Dropbox/Chronostratigraphy/stratigraph


In [2]:
import stratigraph as sg
from mayavi import mlab
import matplotlib.pyplot as plt
import numpy as np
import h5py

# set up graphics:
%matplotlib qt
plt.rcParams['svg.fonttype'] = 'none'

In [19]:
# load data from hdf5 file (this is a 'meanderpy' model)
fname = '/Users/zoltan/Dropbox/Chronostratigraphy/meanderpy_strat_model_example_1.hdf5'
f = h5py.File(fname, 'r')
model  = f['model']
topo = np.array(model['topo'])
strat = np.array(model['strat'])
facies = np.array(model['facies'])
porosity = np.array(model['porosity'])
facies_code = {}
facies_code[int(np.array(model['point bar']))] = 'point bar'
facies_code[int(np.array(model['levee']))] = 'levee'
dx = float(np.array(model['dx']))
f.close()

In [20]:
# create facies volume (the facies data that comes from meanderpy is a 1D array and we need to expand it to 3D)
facies3d = np.zeros((strat.shape[0], strat.shape[1], strat.shape[2]-1))
for i in range(len(facies)):
    facies3d[:,:,i] = facies[i] - 1 # facies codes should start at 0

## Simple block diagram, facies mode

In [9]:
mlab.figure(bgcolor = (1,1,1))
ve = 5.0
scale = 1
strat_switch = 1
layers_switch = 1
contour_switch = 0
dx = 10.0
bottom = np.min(strat) - 2
colors = [[0.9,0.9,0],[0.5,0.25,0]]
line_thickness = 0.5
gap = 50
color_mode = 'facies'
h = 6.0
export = 0
topo_min = np.min(topo[:,:,-1])
topo_max = np.max(topo[:,:,-1])
ci = 1.0 # contour interval

sg.create_exploded_view(strat, None, facies3d, x0=0, y0=0, nx=1, ny=1, 
            gap=gap, dx=dx, ve=ve, scale=scale, plot_strat=True, plot_surfs=True, plot_contours=False, 
            plot_sides=True, color_mode = color_mode, colors=colors, colormap='viridis', line_thickness=line_thickness, 
            bottom=bottom, export=0, topo_min=topo_min, topo_max=topo_max, ci=ci, opacity=1.0)

100%|██████████| 130/130 [00:25<00:00,  5.06it/s]

block 1 done, out of 1 blocks


## Exploded view with 2 x 2 blocks, facies mode

In [11]:
# display 4 blocks, 2 in each direction (this takes a while)
mlab.figure(bgcolor = (1,1,1))
sg.create_exploded_view(strat, None, facies3d, x0=0, y0=0, nx=2, ny=2, 
            gap=gap, dx=dx, ve=ve, scale=scale, plot_strat=True, plot_surfs=True, plot_contours=False, 
            plot_sides=True, color_mode = color_mode, colors=colors, colormap='viridis', line_thickness=line_thickness, 
            bottom=bottom, export=0, topo_min=topo_min, topo_max=topo_max, ci=ci, opacity=1.0)

100%|██████████| 130/130 [00:55<00:00,  2.33it/s]


block 1 done, out of 4 blocks


100%|██████████| 130/130 [00:26<00:00,  4.82it/s]


block 2 done, out of 4 blocks


100%|██████████| 130/130 [00:40<00:00,  3.24it/s]


block 3 done, out of 4 blocks


100%|██████████| 130/130 [00:58<00:00,  2.24it/s]

block 4 done, out of 4 blocks


## Fence diagram, facies mode

In [12]:
mlab.figure(bgcolor = (1,1,1))
sg.create_fence_diagram(strat, None, facies3d, x0=0, y0=0, nx=2, ny=2, dx=dx, ve=ve, scale=scale, 
                        plot_surfs=True, plot_sides=True, color_mode=color_mode, 
                     colors=colors, colormap = 'viridis', line_thickness=line_thickness, bottom=bottom, 
                        export=0, opacity=1.0)

100%|██████████| 4/4 [00:35<00:00,  8.79s/it]


## Exploded view with 2 x 2 blocks, property mode

In [13]:
mlab.figure(bgcolor = (1,1,1))
sg.create_exploded_view(strat, porosity, facies3d, x0=0, y0=0, nx=2, ny=2, 
            gap=gap, dx=dx, ve=ve, scale=scale, plot_strat=True, plot_surfs=True, plot_contours=False, 
            plot_sides=True, color_mode = 'property', colors=colors, colormap='viridis', 
            line_thickness=line_thickness, 
            bottom=bottom, export=0, topo_min=topo_min,topo_max=topo_max, ci=ci, opacity=1.0)

100%|██████████| 130/130 [00:22<00:00,  5.74it/s]


block 1 done, out of 4 blocks


100%|██████████| 130/130 [00:33<00:00,  3.85it/s]


block 2 done, out of 4 blocks


100%|██████████| 130/130 [00:45<00:00,  2.84it/s]


block 3 done, out of 4 blocks


100%|██████████| 130/130 [01:06<00:00,  1.95it/s]

block 4 done, out of 4 blocks


## Fence diagram, facies mode

In [14]:
mlab.figure(bgcolor = (1,1,1))
sg.create_fence_diagram(strat, porosity, facies3d, x0=0, y0=0, nx=2, ny=2, dx=dx, ve=ve, scale=scale, 
                        plot_surfs=True, plot_sides=True, color_mode='property', 
                     colors=colors, colormap = 'viridis', line_thickness=line_thickness, bottom=bottom, 
                        export=0, opacity=1.0)

100%|██████████| 4/4 [00:44<00:00, 11.10s/it]


## Time-elevation curve ('Barrell plot')

In [5]:
# create time array (assumes that every point bar - overbank couplet was deposited in 5 years)
dt = 5.0
time = np.linspace(0, np.round(dt*(topo.shape[2]-1)/3), int((topo.shape[2]-1)/3)+1)

In [6]:
# only consider point bar - overbank couplets so that surfaces represent constant time increments:
elevation = topo[200, 200, ::3] 
fig = sg.plot_strat_diagram(time, elevation, 'years', 'm', max(time), 2.0)

## Chronostratigraphic (Wheeler) diagram

In [21]:
np.shape(strat)

(423, 363, 131)

In [23]:
np.shape(strat)

(423, 363, 66)

In [22]:
# use every third surface in the topography array and every second surface in the stratigraphy array:
strat, wheeler, wheeler_strat, vacuity = sg.create_wheeler_diagram(topo[:,:,::3])

In [8]:
# strike section
plt.figure(figsize=(15,10))
plt.imshow(wheeler[:, 100, :].T, cmap='RdBu', vmin = -8, vmax = 8, extent = [0, dx*strat.shape[0], time[-1], 0],
           interpolation='none', aspect='auto')
plt.gca().invert_yaxis()
plt.colorbar()
plt.xlabel('distance (m)', fontsize = 14)
plt.ylabel('time (years)', fontsize = 14)
plt.title('chronostratigraphic diagram', fontsize = 14)
plt.tight_layout()

In [9]:
# strike section, showing only what is preserved
plt.figure(figsize=(15,10))
plt.imshow(wheeler_strat[:, 100, :].T, cmap='RdBu', vmin = -8, vmax = 8, extent = [0, dx*strat.shape[0], time[-1], 0],
           interpolation='none', aspect='auto')
plt.gca().invert_yaxis()
plt.colorbar()
plt.xlabel('distance (m)', fontsize = 14)
plt.ylabel('time (years)', fontsize = 14)
plt.title('chronostratigraphic diagram', fontsize = 14)
plt.tight_layout()

In [10]:
# dip section
plt.figure(figsize=(15,10))
plt.imshow(wheeler[200, :, :].T, cmap='RdBu', vmin = -8, vmax = 8, extent = [0, dx*strat.shape[0], time[-1], 0],
           interpolation='none', aspect='auto')
plt.gca().invert_yaxis()
plt.colorbar()
plt.xlabel('distance (m)', fontsize = 14)
plt.ylabel('time (years)', fontsize = 14)
plt.title('chronostratigraphic diagram', fontsize = 14)
plt.tight_layout()

In [11]:
# time section
plt.figure(figsize=(15,10))
plt.imshow(wheeler[:, :, 60], cmap='RdBu', vmin = -8, vmax = 8, extent = [0, dx*strat.shape[1], 0, dx*strat.shape[0]],
           interpolation='none')
plt.colorbar()
plt.xlabel('distance (m)', fontsize = 14)
plt.ylabel('distance (m)', fontsize = 14)
plt.title('chronostratigraphic diagram', fontsize = 14)
plt.axis('equal')
plt.tight_layout()

In [12]:
# 3D visualization with isosurface
mlab.figure(bgcolor = (1,1,1))
source = mlab.pipeline.scalar_field(np.swapaxes(wheeler, 0, 1))
source.spacing = [1,1,2]
mlab.pipeline.iso_surface(source, contours=[-1, 1], opacity=1, colormap='RdBu', vmin = -1.5, vmax = 1.5);

In [13]:
# 3D visualization with plane widgets
mlab.figure()
source = mlab.pipeline.scalar_field(np.swapaxes(wheeler, 0, 1))
source.spacing = [1,1,4]
for axis in ['x', 'y', 'z']:
    plane = mlab.pipeline.image_plane_widget(source, plane_orientation = '{}_axes'.format(axis),
                                           slice_index=i, colormap='RdBu', vmin = -8, vmax = 8);

## Cross sections (no need for Mayavi)

In [24]:
# load data from hdf5 file (this is a 'meanderpy' model); it needs to be reloaded as 
# running 'sg.create_wheeler_diagram' has changed the dimensions of the 'strat' array but it did not change
# the 'facies3d' array
fname = '/Users/zoltan/Dropbox/Chronostratigraphy/meanderpy_strat_model_example_1.hdf5'
f = h5py.File(fname, 'r')
model  = f['model']
topo = np.array(model['topo'])
strat = np.array(model['strat'])
facies = np.array(model['facies'])
porosity = np.array(model['porosity'])
facies_code = {}
facies_code[int(np.array(model['point bar']))] = 'point bar'
facies_code[int(np.array(model['levee']))] = 'levee'
dx = float(np.array(model['dx']))
f.close()

In [25]:
fig = sg.plot_model_cross_section_EW(strat, porosity, facies3d, dx, 260, color_mode='facies', 
                                     flattening_ind = False, ve = 5, list_of_colors = ['yellow', 'brown'])

100%|██████████| 130/130 [00:00<00:00, 346.20it/s] 


In [18]:
fig = sg.plot_model_cross_section_NS(strat, porosity, facies3d, dx, 160, color_mode='facies', 
                                     flattening_ind = False, ve = 10, list_of_colors = ['yellow', 'brown'])

100%|██████████| 130/130 [00:00<00:00, 388.05it/s] 


## XES 02 Experiment

In [26]:
import os
dirname = '/Users/zoltan/Dropbox/Chronostratigraphy/XES_02/final_topography_data/topography/'
T = np.zeros((261,111,101))
filenames = os.listdir(dirname)
for filename in filenames:
    surf_no = int(filename[8:-4])
    xyz = np.loadtxt(dirname+filename)
    z = xyz[:,2]
    T[:,:,surf_no-1] = np.reshape(z,(261,111))
dirname = '/Users/zoltan/Dropbox/Chronostratigraphy/XES_02/final_topography_data/basement_topography/'
B = np.zeros((261,111,101))
filenames = os.listdir(dirname)
for filename in filenames:
    surf_no = int(filename[:-4])
    xyz = np.loadtxt(dirname+filename)
    z = xyz[:,2]
    B[:,:,surf_no-1] = np.reshape(z,(261,111))

# np.shape(T)

In [56]:
import pandas as pd
df = pd.read_csv('/Users/zoltan/Dropbox/Chronostratigraphy/XES_02/final_topography_data/sealevel_and_scantimes.csv')
df[:10]

,Scan number,sl(mm),run time (hhh:mm:ss)
0,1,-129.9,0: 0: 0
1,2,-130.1,2: 6: 0
2,3,-130.5,2: 7: 0
3,4,-130.0,10: 0: 0
4,5,-130.2,18: 1: 8
5,7,-135.6,34: 0: 38
6,8,-151.7,41: 59: 59
7,9,-171.8,49: 10: 44
8,10,-200.3,58: 0: 9
9,11,-222.4,66: 0: 26


In [57]:
def convert_to_seconds(string):
    h = int(string.split(':')[0])
    m = int(string.split(':')[1])
    s = int(string.split(':')[2])
    return h*60*60 + m*60 + s

exp_time = np.nan * np.ones((T.shape[2],))
sea_level = np.nan * np.ones((T.shape[2],))

for i in range(len(exp_time)):
    if len(df.loc[df['Scan number'] == i+1]) > 0:
        exp_time[i] = convert_to_seconds(df.loc[df['Scan number'] == i+1]['run time (hhh:mm:ss)'].values[0])
        sea_level[i] = df.loc[df['Scan number'] == i+1]['sl(mm)']
        
        
missing_times = np.where(np.isnan(exp_time)==1)[0]
for i in missing_times:
    exp_time[i] = (exp_time[i-1] + exp_time[i+1]) * 0.5
    sea_level[i] = (sea_level[i-1] + sea_level[i+1]) * 0.5

exp_time = np.delete(exp_time, np.array([2, 5, 12, 25, 30, 31, 48, 60])) # get rid of locations with missing data
sea_level = np.delete(sea_level, np.array([2, 5, 12, 25, 30, 31, 48, 60])) # get rid of locations with missing data
    

plt.figure(figsize=(10, 6))
plt.plot(exp_time, sea_level, 'o-')
plt.xlabel('time (seconds)', fontsize = 16)
plt.ylabel('sea level (mm)', fontsize = 16);

T = np.delete(T, np.array([2, 5, 12, 25, 30, 31, 48, 60]), axis=2) # get rid of locations with no data
B = np.delete(B, np.array([2, 5, 12, 25, 30, 31, 48, 60]), axis=2) # get rid of locations with no data

In [58]:
# create resampled and smooth sea level curve
sampling_rate = 900 # resample at every 900 seconds
time1, sea_level_rs1 = sg.resample_elevation_spl(exp_time, sea_level, sampling_rate)
plt.figure()
plt.plot(exp_time, sea_level, '.-')
plt.plot(time1, sea_level_rs1, '.-')

time2, sea_level_rs2 = sg.resample_elevation_int1d(exp_time, sea_level, sampling_rate)
plt.plot(time2, sea_level_rs2, '.-')

# sea_level_rs1[804:868] = sea_level_rs2[804:868] # replace sea level 1 w/ sea level 2
# sea_level_rs1[972:1215] = sea_level_rs2[972:1215] # replace sea level 1 w/ sea level 2
# sea_level_rs1[267:289] = sea_level_rs2[267:289] # replace sea level 1 w/ sea level 2
# sea_level_rs1[324:405] = sea_level_rs2[324:405] # replace sea level 1 w/ sea level 2
sea_level_rs1[535:578] = sea_level_rs2[535:578] # replace sea level 1 w/ sea level 2
sea_level_rs1[648:810] = sea_level_rs2[648:810] # replace sea level 1 w/ sea level 2
plt.plot(time1, sea_level_rs1, 'b.-')

time = time1.copy()
sea_level_rs = sea_level_rs1.copy()

In [59]:
# resample topography and subsidence arrays:
topo = np.zeros((T.shape[0], T.shape[1], len(time)))
for j in range(T.shape[1]):
    for i in range(T.shape[0]):
        elevation = T[i, j, :].copy()
        time, elevation = sg.resample_elevation_int1d(exp_time, elevation, sampling_rate)
        topo[i,j,:] = elevation  
subsid = np.zeros((B.shape[0], B.shape[1], len(time)))
for j in range(B.shape[1]):
    for i in range(B.shape[0]):
        elevation = B[i, j, :].copy()
        time, elevation = sg.resample_elevation_int1d(exp_time, elevation, sampling_rate)
        subsid[i,j,:] = elevation

# adjust topographic and subsidence surfaces in the proximal corners (for aestethic reasons only)
inds = np.indices(np.shape(subsid[:,:,0]))
inds2 = np.argwhere(inds[0] < -4.8*inds[1] + 120)
inds3 = np.argwhere(inds[0] > 5.13*inds[1] + 142)
for i in range(np.shape(subsid)[2]):
    subsid[:, :, i][inds2[:,0], inds2[:,1]] = -229.6
    subsid[:, :, i][inds3[:,0], inds3[:,1]] = -229.6
for i in range(np.shape(topo)[2]):
    topo[:, :, i][inds2[:,0], inds2[:,1]] = 0
    topo[:, :, i][inds3[:,0], inds3[:,1]] = 0
    
# account for subsidence:
topo_s = topo.copy() 
for i in range(0, topo.shape[2]):
    topo_s[:,:,i] = topo_s[:,:,i]+(subsid[:,:,-1]-subsid[:,:,i])

In [60]:
# QC topographic surfaces (corrected for subsidence)
plt.figure(figsize=(18, 10))
for i in range(0, topo_s.shape[2], 20):
    plt.plot(topo_s[130, :, i], 'k', linewidth=0.5)

In [61]:
# create a number of time-elevation plots, from proximal to distal locations
fig = sg.plot_strat_diagram(time, topo_s[125, 20, :], 'seconds', 'mm', time[-1], np.max(topo_s[125, 20, :]))
fig = sg.plot_strat_diagram(time, topo_s[125, 60, :], 'seconds', 'mm', time[-1], np.max(topo_s[125, 60, :]))
fig = sg.plot_strat_diagram(time, topo_s[125, 100, :], 'seconds', 'mm', time[-1], np.max(topo_s[125, 100, :]))

In [62]:
# convert topographic surfaces to stratigraphy
strat = sg.topostrat(topo_s)

In [63]:
# create wheeler diagram(s)
strat, wheeler, wheeler_strat, vacuity = sg.create_wheeler_diagram(topo_s)

In [65]:
plt.figure(figsize=(20,15))
plt.imshow(wheeler[125,:,:].T, extent = [0, 111, time[-1], 0], cmap='RdBu', vmin = -4, vmax = 4, 
           interpolation='none', aspect='auto')
plt.gca().invert_yaxis()
plt.colorbar();
sl = 20 * (sea_level_rs - np.min(sea_level_rs))/213.5 # sea level curve
plt.plot(2 +np.max(sl)-sl, time, 'k', linewidth=3)
plt.ylim(0, time[-1])
plt.xlim(2, 111)
plt.title('chronostratigraphic diagram', fontsize=20);

In [66]:
plt.figure(figsize=(20,15))
plt.imshow(wheeler_strat[125,:,:].T, extent = [0, 111, time[-1], 0], cmap='RdBu', vmin = -4, vmax = 4, 
           interpolation='none', aspect='auto')
plt.gca().invert_yaxis()
plt.colorbar();
sl = 20 * (sea_level_rs - np.min(sea_level_rs))/213.5
plt.plot(2 +np.max(sl)-sl, time, 'k', linewidth=3)
plt.ylim(0, time[-1])
plt.xlim(2, 111)
plt.title('chronostratigraphic diagram w/ vacuity', fontsize=20);

In [67]:
# 3D visualization with isosurface
mlab.figure()
source = mlab.pipeline.scalar_field(np.swapaxes(wheeler, 0, 1))
source.spacing = [5, 1, 0.1]
mlab.pipeline.iso_surface(source, contours=[-2, 2], opacity=0.5, colormap='RdBu', vmin = -4, vmax = 4);

In [68]:
# 3D visualization with plane widgets
mlab.figure()
source = mlab.pipeline.scalar_field(np.swapaxes(wheeler, 0, 1))
source.spacing = [5, 1, 0.2]
for axis in ['x', 'y', 'z']:
    plane = mlab.pipeline.image_plane_widget(source, plane_orientation = '{}_axes'.format(axis),
                                           slice_index=i, colormap='RdBu', vmin = -4, vmax = 4);

In [69]:
# create 3D facies array (as a function of water depth in this case)
ny, nx, nz = np.shape(strat)
facies = np.zeros((ny, nx, nz))
for i in range(facies.shape[2]):
    topo_sl = topo[:, :, i] - sea_level_rs[i]
    facies_sl = np.zeros(np.shape(topo_sl))
    facies_sl[topo_sl >= 0] = 0
    facies_sl[(topo_sl < 0) & (topo_sl >= -100)] = 1
    facies_sl[topo_sl < -100] = 2
    facies[:, :, i] = facies_sl

/Users/zoltan/anaconda3/envs/meanderpy/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in greater_equal
  import sys
/Users/zoltan/anaconda3/envs/meanderpy/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in less
  
/Users/zoltan/anaconda3/envs/meanderpy/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in greater_equal
  
/Users/zoltan/anaconda3/envs/meanderpy/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in less
  if __name__ == '__main__':


In [70]:
# eliminating nans from 'strat' array
for i in range(np.shape(strat)[2]):
    t = strat[:, :, i]
    t[np.isnan(t) == 1] = t[136, 1]
    strat[:, :, i] = t

In [71]:
# plot cross section colored by water depth (= facies in this case)
fig = sg.plot_model_cross_section_EW(strat, facies, facies, dx = 50, xsec = 135, color_mode = 'facies', 
                                     map_aspect = 0.2,
                                     line_freq = 5,
                                     flattening_ind = False, units = 'mm')

100%|██████████| 1186/1186 [00:23<00:00, 50.27it/s]


In [72]:
fig = sg.plot_model_cross_section_NS(strat, facies, facies, dx = 10, xsec = 50, color_mode = 'facies',
                                     map_aspect = 0.2,
                                     line_freq = 5,
                                     flattening_ind = False, units = 'mm')

100%|██████████| 1186/1186 [00:17<00:00, 67.68it/s]
